In [21]:
%pip install datasets
%pip install numpy
%pip install tabulate
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from tabulate import tabulate
import json
import numpy as np

/Users/shiyue/Documents/code/testeven/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/shiyue/Documents/code/testeven/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load QAPyramid Data

In [3]:
dataset = load_dataset("shiyue/QAPyramid")["cnndm_test"]

In [4]:
dataset

Dataset({
    features: ['example_id', 'source', 'reference', 'QAs', 'QA_labels', 'system_outputs', 'system_scores'],
    num_rows: 500
})

In [5]:
dataset[0]

{'example_id': '000571afe702684d90c1d222ce70b1e1375c1016',
 'source': "Club Tijuana star Juan Arango conjured memories Luis Suarez in his team's 4-3 defeat by Monterrey in the Mexican league - but it was not through prodigious scoring. The Venezuelan icon Arango sank his teeth into the shoulder of Jesus Zavela as his temper flared in the defeat. He was not booked by the referee but could face a heavy retrospective ban. Juan Arango (left) bites the shoulder of opponent Jesus Zavela in a moment of madness . Zavala holds his shoulder after being bitten by Arango, in the game Zavala's side won 4-3 in Mexico . Zavala shows the referee the mark on his shoulder after being bittern by Arango . Arango (right) earlier scored a magnificent free kick to bring his Club Tijuana team level against Monterrey . Arango had earlier curled in a magnificent free kick for his team to bring them level after falling 2-0 down early on in the encounter. But the 34-year-old overshadowed his goal with the bite as

## Get the 50 examples th with QA labels 

In [6]:
subset = [example for example in dataset if example["QA_labels"]["bart"]]
len(subset)

50

## Calculate the average metric scores of each system

In [7]:
systems = [system for system in subset[0]["system_scores"]]
systems

['bart',
 'pegasus',
 'brio',
 'brio-ext',
 'matchsum',
 'mixtral-8x22b-instruct-v0.1',
 'llama-3-8b-instruct',
 'GPT4',
 'llama-3-70b-instruct',
 'mixtral-8x7b-instruct-v0.1']

In [8]:
finetuned_systems = ['bart', 'pegasus', 'brio', 'brio-ext', 'matchsum']
llm_systems = ['llama-3-8b-instruct', 'llama-3-70b-instruct', 
               'mixtral-8x7b-instruct-v0.1', 'mixtral-8x22b-instruct-v0.1', 'GPT4']

In [36]:
metrics = [metric for metric in subset[0]["system_scores"]["bart"]]
metrics

['QAPyramid',
 'nQAPyramid',
 'p_r',
 'p_l',
 'R1_recall',
 'R1_f1',
 'R2_recall',
 'R2_f1',
 'RL_recall',
 'RL_f1',
 'ACU',
 'nACU',
 'BERTScore_recall',
 'BERTScore_f1',
 'BARTScore',
 'Lite3Pyramid',
 'Lite2Pyramid_w_ACU',
 'AutoACU_recall',
 'AutoACU_f1',
 'METEOR',
 'ChrF',
 'SemiAutoACU',
 'GEval',
 'AutoQAPyramid',
 'SemiAutoQAPyramid',
 'nAutoQAPyramid',
 'nSemiAutoQAPyramid']

In [9]:
human_metrics = ['ACU', 'nACU', 'QAPyramid', 'nQAPyramid']
semi_auto_metrics = ['SemiAutoACU', 'Lite2Pyramid_w_ACU', 'SemiAutoQAPyramid', 'nSemiAutoQAPyramid']
auto_metrics = ['R1_recall', 'R1_f1', 'R2_recall', 'R2_f1', 'RL_recall', 'RL_f1', 'METEOR', 'ChrF', 'BERTScore_recall',
                'BERTScore_f1', 'BARTScore', 'GEval', 'AutoACU_recall', 'AutoACU_f1', 'Lite3Pyramid', 'AutoQAPyramid',
                'nAutoQAPyramid']

In [82]:
rows = []
metric_scores_map = {}
for metric in human_metrics + semi_auto_metrics + auto_metrics:
    metric_scores = []
    for system in finetuned_systems + llm_systems:
        if metric in subset[0]["system_scores"][system]:
            score = np.mean([example["system_scores"][system][metric] for example in subset])
        else:
            score = "-"
        metric_scores.append(score)
    metric_scores_map[metric] = metric_scores
    metric_scores = [metric] + list(map(lambda x: round(x, 2) if x != "-" else x, metric_scores))
    rows.append(metric_scores)
headers = ['System'] + finetuned_systems + ['llama3-8b', 'llama3-70b', 'mixtral-8x7b', 'mixtral-8x22b', 'GPT4']

In [83]:
print(tabulate(rows, headers=headers))

System                bart    pegasus    brio    brio-ext    matchsum  llama3-8b    llama3-70b    mixtral-8x7b    mixtral-8x22b    GPT4
------------------  ------  ---------  ------  ----------  ----------  -----------  ------------  --------------  ---------------  ------
ACU                   0.37       0.35    0.43        0.41        0.41  -            -             -               -                -
nACU                  0.29       0.3     0.35        0.32        0.31  -            -             -               -                -
QAPyramid             0.51       0.46    0.56        0.55        0.5   0.54         0.53          0.48            0.48             0.55
nQAPyramid            0.47       0.44    0.53        0.52        0.46  0.4          0.47          0.45            0.44             0.46
SemiAutoACU           0.36       0.35    0.43        0.41        0.38  0.38         0.35          0.31            0.37             0.37
Lite2Pyramid_w_ACU    0.45       0.43    0.5        

## Meta-evalution: get the correlations between QAPyramid and other metrics

### system-level correlation

In [84]:
from scipy import stats

metrics = ['ACU', 'SemiAutoACU', 'Lite2Pyramid_w_ACU', 'SemiAutoQAPyramid', 'R1_recall', 
           'R2_recall', 'RL_recall', 'METEOR', 'ChrF', 'BERTScore_recall', 'BARTScore', 
           'GEval', 'AutoACU_recall', 'Lite3Pyramid', 'AutoQAPyramid']

In [85]:
QAPyramid_scores = metric_scores_map["QAPyramid"]
rows = []
for metric in metrics:
    metric_scores = metric_scores_map[metric]
    corr_ft = str(round(stats.kendalltau(QAPyramid_scores[:5], metric_scores[:5]).statistic, 3))
    corr_llm, corr_all = '-', '-'
    if metric != 'ACU':
        corr_llm = str(round(stats.kendalltau(QAPyramid_scores[5:], metric_scores[5:]).statistic, 3))
        corr_all = str(round(stats.kendalltau(QAPyramid_scores, metric_scores).statistic, 3))
    rows.append([metric, corr_ft, corr_llm, corr_all])
print(tabulate(rows, headers=['Metric', 'correlation (FT)', 'correlation (LLM)', 'correlation (All)']))

Metric                correlation (FT)  correlation (LLM)    correlation (All)
------------------  ------------------  -------------------  -------------------
ACU                                0.8  -                    -
SemiAutoACU                        0.8  0.2                  0.556
Lite2Pyramid_w_ACU                 0.8  0.2                  0.467
SemiAutoQAPyramid                  1    0.8                  0.956
R1_recall                          0.8  0.4                  0.556
R2_recall                          0.6  -0.2                 0.156
RL_recall                         -0.2  0.2                  0.156
METEOR                             0.8  -0.2                 0.2
ChrF                               0.8  -0.2                 0.289
BERTScore_recall                   0.8  0.2                  0.289
BARTScore                          0.8  0.2                  0.467
GEval                              0.6  0.0                  0.422
AutoACU_recall                     0.6  0.

## summary-level correlation

In [94]:
corr_fts = {metric: [] for metric in metrics}
corr_llms = {metric: [] for metric in metrics}
corr_alls = {metric: [] for metric in metrics}
for example in subset:
    metric_scores_map = {}
    for metric in ["QAPyramid"] + metrics:
        metric_scores = []
        for system in finetuned_systems + llm_systems:
            if metric in subset[0]["system_scores"][system]:
                score = example["system_scores"][system][metric]
            else:
                score = "-"
            metric_scores.append(score)
        metric_scores_map[metric] = metric_scores
    QAPyramid_scores = metric_scores_map["QAPyramid"]
    for metric in metrics:
        metric_scores = metric_scores_map[metric]
        corr_ft = stats.kendalltau(QAPyramid_scores[:5], metric_scores[:5]).statistic
        corr_llm, corr_all = '-', '-'
        if metric != 'ACU':
            corr_llm = stats.kendalltau(QAPyramid_scores[5:], metric_scores[5:]).statistic
            corr_all = stats.kendalltau(QAPyramid_scores, metric_scores).statistic
        corr_fts[metric].append(corr_ft if corr_ft == "-" or not np.isnan(corr_ft) else 0)
        corr_llms[metric].append(corr_llm if corr_llm == "-" or not np.isnan(corr_llm) else 0)
        corr_alls[metric].append(corr_all if corr_all == "-" or not np.isnan(corr_all) else 0) 
rows = []
for metric in metrics:
    rows.append([metric, round(np.mean(corr_fts[metric]), 3), 
                 round(np.mean(corr_llms[metric]), 3) if metric != "ACU" else "-", 
                 round(np.mean(corr_alls[metric]), 3) if metric != "ACU" else "-"])
print(tabulate(rows, headers=['Metric', 'correlation (FT)', 'correlation (LLM)', 'correlation (All)']))

Metric                correlation (FT)  correlation (LLM)    correlation (All)
------------------  ------------------  -------------------  -------------------
ACU                              0.435  -                    -
SemiAutoACU                      0.508  0.35                 0.476
Lite2Pyramid_w_ACU               0.503  0.501                0.564
SemiAutoQAPyramid                0.603  0.537                0.63
R1_recall                        0.459  0.437                0.5
R2_recall                        0.445  0.378                0.398
RL_recall                        0.351  0.354                0.38
METEOR                           0.448  0.293                0.405
ChrF                             0.461  0.293                0.423
BERTScore_recall                 0.471  0.384                0.449
BARTScore                        0.47   0.459                0.507
GEval                            0.289  0.279                0.366
AutoACU_recall                   0.444  0.39